In [ ]:
# ==============================================================================
# NLP-SNN BENCHMARKING RUNNER (v7.1 - Advanced Encoders & Early Stopping)
# ==============================================================================

%run nlp_snn_fun107.py
import sys
import subprocess
import os
import traceback
import warnings

# Suppress minor TF warnings for cleaner output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

# 1. CRITICAL DEPENDENCY CHECK
try:
    import tensorflow_text
    print("[System] tensorflow_text is already installed.")
except ImportError:
    print("[System] tensorflow_text not found. Installing now...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "tensorflow-text"])
        import tensorflow_text
        print("[System] Successfully installed tensorflow_text.")
    except Exception as e:
        print(f"[System] WARNING: Automatic installation failed ({e}).")
        print("          Some models (BERT, USE, mmBERT) may fail.")

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 2. IMPORT BENCHMARKING FUNCTIONS
try:
    from nlp_snn_fun107 import train_benchmark
except ImportError:
    print("\n[CRITICAL ERROR] Could not import 'nlp_snn_benchmarking_functions_50.py'.")
    print("Please ensure the functions file is uploaded and in the same directory.")
    sys.exit(1)

# ==============================================================================
# CONFIGURATION
# ==============================================================================

datasets = ['imdb', 'ag_news', 'sst2', 'yelp']

# FULL ENCODER SUITE (Updated with Advanced Transformers)
encoders = [
    'DAN',          # Baseline: Deep Averaging Network
    'CNN',          # Baseline: 1D Convolution
    'Transformer',  # Baseline: Custom Attention
    'Word2Vec',     # Static: NNLM-128
    'USE',          # Transfer: Universal Sentence Encoder
    'BERT',         # Advanced: Official BERT-Base
    'RoBERTa',      # Advanced: Robustly Optimized BERT
    'NeoBERT',      # Advanced: Electra (Generator-Discriminator)
    'mmBERT'        # Advanced: Multi-lingual BERT
]

# SNN CONFIGURATION
SNN_EPOCHS = 25
EARLY_STOP_PATIENCE = 5

# ==============================================================================
# MAIN EXECUTION LOOP
# ==============================================================================

def main():
    all_metrics = []

    print("############################################################")
    print("STARTING NLP-SNN BENCHMARK (v7.1)")
    print(f"Datasets ({len(datasets)}): {datasets}")
    print(f"Encoders ({len(encoders)}): {encoders}")
    print(f"SNN Scheme: 1-SADP | Max Epochs: {SNN_EPOCHS} | Patience: {EARLY_STOP_PATIENCE}")
    print("############################################################")

    for data_name in datasets:
        for enc_name in encoders:
            try:
                print(f"\n>>> RUNNING: Dataset={data_name.upper()} | Encoder={enc_name} <<<")

                # Execute the pipeline with early stopping
                logs = train_benchmark(
                    data_name,
                    enc_name,
                    snn_epochs=SNN_EPOCHS,
                    patience=EARLY_STOP_PATIENCE
                )

                # Aggregate results
                for log in logs:
                    all_metrics.append(log)

            except KeyboardInterrupt:
                print("\n[System] Benchmark interrupted by user. Saving current progress...")
                save_and_plot(all_metrics)
                sys.exit(0)
            except Exception as e:
                print(f"!!! CRITICAL FAILURE on {data_name}-{enc_name}: {type(e).__name__}: {e}")

    # Final Report
    save_and_plot(all_metrics)

# ==============================================================================
# ANALYSIS & VISUALIZATION HELPERS
# ==============================================================================

def save_and_plot(metrics_list):
    if not metrics_list:
        print("\n[System] No results to save.")
        return

    # 1. Convert to DataFrame
    df = pd.DataFrame(metrics_list)
    output_csv = "nlp_snn_benchmark_results_v7.csv"
    df.to_csv(output_csv, index=False)
    print(f"\n[System] Benchmark Complete. Raw data saved to {output_csv}")

    # 2. Detailed Tabular Report
    print("\n" + "="*100)
    print("FINAL BENCHMARK REPORT (Metrics at Best/Final Epoch)")
    print("="*100)

    # Filter for the last epoch of each run (which is the best due to early stopping or max epochs)
    final_df = df[df['epoch'] == df.groupby(['dataset', 'encoder'])['epoch'].transform('max')].copy()

    # Select and Rename columns for clarity
    display_cols = [
        'dataset', 'encoder', 'train_acc',
        'time_epoch', 'energy_sample_j',
        'enc_time', 'feat_time'
    ]

    final_table = final_df[display_cols].rename(columns={
        'dataset': 'Dataset',
        'encoder': 'Encoder',
        'train_acc': 'Accuracy',
        'time_epoch': 'SNN Epoch(s)',
        'energy_sample_j': 'Energy (J)',
        'enc_time': 'ANN Train(s)',
        'feat_time': 'Extract(s)'
    })

    # Sort by Dataset then Accuracy (Descending)
    final_table = final_table.sort_values(by=['Dataset', 'Accuracy'], ascending=[True, False])

    # Print formatted table
    print(final_table.to_string(index=False, formatters={
        'Accuracy': '{:.4f}'.format,
        'SNN Epoch(s)': '{:.2f}'.format,
        'Energy (J)': '{:.2e}'.format,
        'ANN Train(s)': '{:.2f}'.format,
        'Extract(s)': '{:.2f}'.format
    }))
    print("="*100)

    # 3. Generate Plots
    try:
        sns.set_theme(style="whitegrid")

        # Plot A: Accuracy Evolution (Line Plot)
        plt.figure(figsize=(14, 8))
        sns.lineplot(
            data=df,
            x='epoch',
            y='train_acc',
            hue='encoder',
            style='dataset',
            markers=True,
            dashes=False,
            linewidth=2
        )
        plt.title(f"SNN (1-SADP) Accuracy Evolution (Max {SNN_EPOCHS} Epochs w/ ES)")
        plt.ylabel("Accuracy")
        plt.xlabel("Epoch")
        plt.ylim(0.4, 1.0)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title="Encoder")
        plt.tight_layout()
        plt.savefig("snn_accuracy_evolution.png", dpi=300)
        print("[System] Saved plot: snn_accuracy_evolution.png")

        # Plot B: Energy Efficiency (Bar Plot)
        if not final_df.empty:
            plt.figure(figsize=(12, 6))
            sns.barplot(
                data=final_df,
                x='encoder',
                y='energy_sample_j',
                hue='dataset',
                palette='viridis'
            )
            plt.title("Estimated On-Chip Dynamic Energy (Joules per Batch)")
            plt.ylabel("Energy (Joules) - Log Scale")
            plt.xlabel("Encoder Architecture")
            plt.yscale('log')
            plt.legend(title="Dataset")
            plt.tight_layout()
            plt.savefig("snn_energy_efficiency.png", dpi=300)
            print("[System] Saved plot: snn_energy_efficiency.png")

    except Exception as e:
        print(f"[System] Error generating plots: {e}")

if __name__ == "__main__":
    main()

[20:00:20] [System] tensorflow_text imported successfully.
[System] tensorflow_text is already installed.
[20:00:21] [System] tensorflow_text imported successfully.
############################################################
STARTING NLP-SNN BENCHMARK (v7.1)
Datasets (4): ['imdb', 'ag_news', 'sst2', 'yelp']
Encoders (9): ['DAN', 'CNN', 'Transformer', 'Word2Vec', 'USE', 'BERT', 'RoBERTa', 'NeoBERT', 'mmBERT']
SNN Scheme: 1-SADP | Max Epochs: 25 | Patience: 5
############################################################

>>> RUNNING: Dataset=IMDB | Encoder=DAN <<<
[2026-02-06 20:00:21] ==================================================================
[2026-02-06 20:00:21] STARTING PIPELINE: Dataset=imdb | Encoder=DAN
[2026-02-06 20:00:21] ==================================================================
[2026-02-06 20:00:21] [Data] Initiating load for dataset: IMDB...
[2026-02-06 20:00:21] [Data] Downloading/Loading TFDS: imdb_reviews...


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.6YG8ID_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.6YG8ID_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
[2026-02-06 20:01:04] [Data] Extracting... (Train Limit: 10000, Test Limit: 10000)
[2026-02-06 20:01:11] [Data] Loaded 10000 train, 10000 test samples.
[2026-02-06 20:01:11] [Encoder] Constructing architecture: DAN
[2026-02-06 20:01:12] [Encoder] Model Built. Feature Output Shape: (None, 256)
[2026-02-06 20:01:12] [Encoder] Starting ANN Pre-training (1 Epoch)...
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.6290 - loss: 0.6104
[2026-02-06 20:01:19] [Encoder] ANN Training Complete. Time: 7.80s | Final Acc: 0.7211
[2026-02-06 20:01:19] [Features] Generating Static Embeddings for SNN...
[2026-02-06 20:01:22] [Features] Extraction Complete. Time: 2.20s
[2026-02-06 20:01:22] [Features] Embedding Shape: (10000, 256)
[2026-02-06 20:01:22] [SNN] Info: Running 1-SADP with Early Stopping (Patience=5)
[2026-02-06 20:01:22] [SNN] Init 1-SADP Arch